<a href="https://colab.research.google.com/github/joshikavita01/streamlit-demo/blob/master/Assigment3_Ques2(b).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from google.colab.patches import cv2_imshow

In [44]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [45]:
!unzip "/content/gdrive/MyDrive/lfw_patch.zip" -d "/content/" 

Streaming output truncated to the last 5000 lines.
  inflating: /content/lfw_patch/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0012.jpg  
  inflating: /content/lfw_patch/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0013.jpg  
  inflating: /content/lfw_patch/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0014.jpg  
  inflating: /content/lfw_patch/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0015.jpg  
  inflating: /content/lfw_patch/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0016.jpg  
  inflating: /content/lfw_patch/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0017.jpg  
  inflating: /content/lfw_patch/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0018.jpg  
  inflating: /content/lfw_patch/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0019.jpg  
  inflating: /content/lfw_patch/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0020.jpg  
  inflating: /content/lfw_patch/Luiz_Inacio_Lula_da_Silva/Luiz_Inacio_Lula_da_Silva_0021.jpg  

In [55]:
import torch.utils.data as data
from PIL import Image
import os
import os.path
def _loader(path):
    img = Image.open(path).convert('L')
    return img
def _list_reader(fileList):
    imgList = []
    with open(fileList, 'r') as file:
        for line in file.readlines():
            imgPath, label = line.strip().split(' ')
            imgList.append((imgPath, int(label)))
    return imgList
class ImageList(data.Dataset):
    def __init__(self, root, fileList, transform=None, list_reader=_list_reader, loader=_loader):
        self.root      = root
        self.imgList   = list_reader(fileList)
        self.transform = transform
        self.loader    = loader

    def __getitem__(self, index):
        imgPath, target = self.imgList[index]
        img = self.loader(os.path.join(self.root, imgPath))

        if self.transform is not None:
            img = self.transform(img)
        return img, target

    def __len__(self):
        return len(self.imgList)

In [56]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class mfm(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, type=1):
        super(mfm, self).__init__()
        self.out_channels = out_channels
        if type == 1:
            self.filter = nn.Conv2d(in_channels, 2*out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        else:
            self.filter = nn.Linear(in_channels, 2*out_channels)

    def forward(self, x):
        x = self.filter(x)
        out = torch.split(x, self.out_channels, 1)
        return torch.max(out[0], out[1])

class group(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(group, self).__init__()
        self.conv_a = mfm(in_channels, in_channels, 1, 1, 0)
        self.conv   = mfm(in_channels, out_channels, kernel_size, stride, padding)

    def forward(self, x):
        x = self.conv_a(x)
        x = self.conv(x)
        return x

class resblock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(resblock, self).__init__()
        self.conv1 = mfm(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = mfm(in_channels, out_channels, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        res = x
        out = self.conv1(x)
        out = self.conv2(out)
        out = out + res
        return out

     

class network_29layers(nn.Module):
    def __init__(self, block, layers, num_classes=79077):
        super(network_29layers, self).__init__()
        self.conv1  = mfm(1, 48, 5, 1, 2)
        self.pool1  = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.block1 = self._make_layer(block, layers[0], 48, 48)
        self.group1 = group(48, 96, 3, 1, 1)
        self.pool2  = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.block2 = self._make_layer(block, layers[1], 96, 96)
        self.group2 = group(96, 192, 3, 1, 1)
        self.pool3  = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.block3 = self._make_layer(block, layers[2], 192, 192)
        self.group3 = group(192, 128, 3, 1, 1)
        self.block4 = self._make_layer(block, layers[3], 128, 128)
        self.group4 = group(128, 128, 3, 1, 1)
        self.pool4  = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        self.fc     = mfm(8*8*128, 256, type=0)
        self.fc2    = nn.Linear(256, num_classes)
            
    def _make_layer(self, block, num_blocks, in_channels, out_channels):
        layers = []
        for i in range(0, num_blocks):
            layers.append(block(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)

        x = self.block1(x)
        x = self.group1(x)
        x = self.pool2(x)

        x = self.block2(x)
        x = self.group2(x)
        x = self.pool3(x)

        x = self.block3(x)
        x = self.group3(x)
        x = self.block4(x)
        x = self.group4(x)
        x = self.pool4(x)

        x = x.view(x.size(0), -1)
        fc = self.fc(x)
        fc = F.dropout(fc, training=self.training)
        out = self.fc2(fc)
        return out, fc


def LightCNN_29Layers(**kwargs):
    model = network_29layers(resblock, [1, 2, 3, 4], **kwargs)
    return model



In [57]:
model_path = "/content/gdrive/MyDrive/LightCNN_29Layers_checkpoint.pth.tar"
dataset = "/content/lfw_patch"
imglist = "/content/gdrive/MyDrive/imglist.txt"
save_path = "/content/data"

In [58]:
from __future__ import print_function
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import numpy as np
import cv2




def main():
    m = LightCNN_29Layers(num_classes=79077)

    m.eval()
    m = torch.nn.DataParallel(m).cuda()


    if os.path.isfile(model_path):
        
        checkpoint = torch.load(model_path)
        m.load_state_dict(checkpoint['state_dict'])

    path_list  = _img_path(imglist)
    transform = transforms.Compose([transforms.ToTensor()])
    count     = 0
    input     = torch.zeros(1, 1, 128, 128)
    for img_name in path_list:
        count = count + 1
       
        img   = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
        
        if img.shape != (128, 128):
            img = cv2.resize(img, (128, 128))
        

        img   = np.reshape(img, (128, 128, 1))
        img   = transform(img)
        input[0,:,:,:] = img

        start = time.time()
        
        input = input.cuda()
        input_var   = torch.autograd.Variable(input, volatile=True)
        _, features = m(input_var)

        print(features.data.cpu().numpy()[0])


def _list(list_path):
    img_list = []
    with open(list_path, 'r') as f:
        for line in f.readlines()[1:]:

            img_path = line.strip().split()
            img_list.append(img_path[0])
            
    return img_list


def _img_path(imglist):
  img_list = _list(imglist)
  imgpath = []
  for i in img_list:
    folder_path = os.path.join(dataset, i)
    for img in os.listdir(folder_path):
      img_path = os.path.join(folder_path, img)
      
      imgpath.append(img_path)

  return imgpath


if __name__ == '__main__':
    main()

In [59]:
def calc_hist(path):

      m = LightCNN_29Layers(num_classes=79077)

      m.eval()
      m = torch.nn.DataParallel(m).cuda()


      transform = transforms.Compose([transforms.ToTensor()])
      input     = torch.zeros(1, 1, 128, 128)

      if os.path.isfile(model_path):
        
        checkpoint = torch.load(model_path)
        m.load_state_dict(checkpoint['state_dict'])

      img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
      if img.shape != (128, 128):
            img = cv2.resize(img, (128, 128))
      img   = np.reshape(img, (128, 128, 1))
      
      img   = transform(img)
      input[0,:,:,:] = img

      start = time.time()
        
      input = input.cuda()
      input_var   = torch.autograd.Variable(input, volatile=True)
      _, features = m(input_var)
      
      ff = features.flatten()
      ff = ff.cpu()
      ff = ff.detach().numpy()
      
      hist = cv2.calcHist([ff], [0], None, [256], [0, 256])
    
      return hist

In [60]:
calc_hist('/content/lfw_patch/Christopher_Patten/Christopher_Patten_0001.jpg')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


array([[197.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  0.],
       [  

In [61]:
def eucledean_dist_hist(hist1, hist2):
  sum = 0
  for i in range (0,256):
    sum = sum + (hist1[i][0]-hist2[i][0])**2
  dist = math.sqrt(sum)
  print('euclidean distance:', dist)

In [62]:

hist1 = calc_hist('/content/lfw_patch/Christopher_Patten/Christopher_Patten_0001.jpg')
hist2 = calc_hist('/content/lfw_patch/Christopher_Patten/Christopher_Patten_0001.jpg')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [63]:
eucledean_dist_hist(hist1, hist2)

euclidean distance: 2.0
